In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import torch

# -------------------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------------------
train_df = pd.read_csv("medical_cases_train/medical_cases_train.csv")[["description", "transcription"]].dropna()
val_df = pd.read_csv("medical_cases_validation/medical_cases_validation.csv")[["description", "transcription"]].dropna()
test_df = pd.read_csv("medical_cases_test/medical_cases_test.csv")[["description", "transcription"]].dropna()

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# -------------------------------------------------------------------
# 2. FORMAT PROMPTS
# -------------------------------------------------------------------
def format_prompt(example):
    return {
        "text": f"<start_of_turn>user\n{example['description']}\n<end_of_turn>\n<start_of_turn>model\n{example['transcription']}<end_of_turn>"
    }

train_dataset = train_dataset.map(format_prompt)
val_dataset = val_dataset.map(format_prompt)
test_dataset = test_dataset.map(format_prompt)

# -------------------------------------------------------------------
# 3. LOAD MODEL & TOKENIZER (from Hugging Face Transformers)
# -------------------------------------------------------------------
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_fast=False  # <-- This helps avoid TypeError in some cases
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    load_in_4bit=True,
    device_map="auto"
)
# -------------------------------------------------------------------
# 4. APPLY LoRA using PEFT
# -------------------------------------------------------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

# -------------------------------------------------------------------
# 5. TOKENIZATION
# -------------------------------------------------------------------
def tokenize(example):
    tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

train_dataset = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize, remove_columns=test_dataset.column_names)

# -------------------------------------------------------------------
# 6. TRAINING ARGUMENTS
# -------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./deepseek-lora-medical",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    num_train_epochs=6,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

# -------------------------------------------------------------------
# 7. TRAINER
# -------------------------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

trainer.train()
# -------------------------------------------------------------------
# 9. SAVE MODEL
# -------------------------------------------------------------------
model.save_pretrained("./deepseek-lora-medical")
tokenizer.save_pretrained("./deepseek-lora-medical")


Map:   0%|          | 0/1724 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Map:   0%|          | 0/1724 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

/tmp/ipykernel_656868/3153629755.py:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/opt/anaconda3/envs/ayush/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,4.235000
20,3.318300
30,2.990800
40,2.873600
50,2.671000
60,2.688000
70,2.588200
80,2.489300
90,2.434800
100,2.560300


('./deepseek-lora-medical/tokenizer_config.json',
 './deepseek-lora-medical/special_tokens_map.json',
 './deepseek-lora-medical/tokenizer.json')

In [2]:
torch.cuda.empty_cache()